<a target="_blank" href="https://colab.research.google.com/github/da-python/new-modul-python/blob/main/Lesson3/Magic_White_Powder.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Задание 1.

* Вы - менеджер по продажам в нефтехимической компании. Вы ведете несколько ключевых клиентов, которые закупают продукты вашей компании. Ваша компания запускает новый продукт - "Волшебный белый порошок" (ВБП), который по своим свойствам похож на тот, что уже есть на рынке. Ваши клиенты предпочитают формульный подход к ценообразованию. Вам нужно предложить клиентам формулу расчета цены на ВБП. В зависимости от объемов, которые покупают клиенты, им может быть предложена скидка.


* ВБП производиться из нефти. Один из известных способов производства предполагает что из 16 баррелей нефти сделают одну тонну ВБП, а затраты на производство в районе 400 евро.

### Что нужно сделать?


1. Определить базовую формулу цены на заводе. То есть как бы выглядела цена на исторических данных.
2. Отобразить на графике.
3. Сделать расчет возможной цены по формуле для каждого из клиентов на условиях DDP (цена с доставкой). Записать все в один эксель файл, на разных листах. Каждый лист - название клиента.
4. Предложить формулу цены каждому из клиентов на условиях DDP (цена с доставкой).
- Создать директорию «для клиентов» и в ней сложить  файлы с расчетами.
- Каждый клиент - отдельный файл (любой из docx, xlsx, pptx, pdf)  с именем клиента… 



In [ ]:
!pip install yfinance
!pip install xlsxwriter

In [ ]:
import os
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt 
import xlsxwriter

In [ ]:
crude_oil_ticker = "CL=F"
exr_eurusd_ticker = "EURUSD=X"
crude_oil = yf.download(crude_oil_ticker)
exr = yf.download(exr_eurusd_ticker)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:

# Затраты на производство
PRODUCTION_COST = 400 # (EUR)

# Расходы на логистику
EU_LOGISTIC_COST_EUR = 30 # в Европу в евро
CN_LOGISTIC_COST_USD = 130 # в Китай в долларах

# * Справочная информация по клиентам(объемы, локации, комментарии) 
customers = {
    'Monty':{
        'location':'EU',
        'volumes':200,
        'comment':'moving_average'
    },
    
    'Triangle':{
        'location':'CN',
        'volumes': 30,
        'comment': 'monthly'
    },
    'Stone':{
        'location':'EU',
        'volumes': 150,
        'comment': 'moving_average'
    },
    'Poly':{
        'location':'EU',
        'volumes': 70,
        'comment': 'monthly'
    }
}
# Скидки
discounts = {'99': 0.01, # до 100 тонн 1%
             '299': 0.05, #  до 300 тонн 5%
             '300': 0.1}   # больше 300 тонн 10%


In [ ]:
df = pd.concat([crude_oil.resample('M').mean()['Close'], exr.resample('M').mean()['Close']],axis=1)
df.columns = ['crude_oil_price', 'eureusd']
df = df['2022-01-01':'2022-12-31']
# Formula (16*CrudeOil + 400 ) * Discount + Logistics
df["proccessing_usd"] = df['eureusd'] * 400
df["base_wpb_price_usd"] = df["crude_oil_price"]*16 + df['proccessing_usd']


In [ ]:
def check_volume_discounts(x):
  if 0 < x <= 99:
    return 0.01
  elif 99 < x <= 299:
    return 0.05
  elif x >= 300:
    return 0.1

In [ ]:
print(f" 50 {check_volume_discounts(50)}")
print(f" 150 {check_volume_discounts(150)}")
print(f" 350 {check_volume_discounts(350)}")

 50 0.01
 150 0.05
 350 0.1


In [ ]:

xlfilepath = 'wbp_prices.xlsx'
with pd.ExcelWriter(xlfilepath, engine='xlsxwriter') as writer:
  for client_name, client_info in customers.items():
    client_df = df.copy()
    
    if client_info.get('location') == 'EU':
      client_df['logistics'] = EU_LOGISTIC_COST_EUR
      client_df['logistics'] = client_df['eureusd'] * EU_LOGISTIC_COST_EUR
    elif client_info.get('location') == 'CN':
      client_df['logistics'] = CN_LOGISTIC_COST_USD
    
    if client_info.get('volumes'):
      client_df['volumes'] = client_info.get('volumes')

    client_df['discount'] = client_df['volumes'].apply(check_volume_discounts)
    client_df['WBP_price_usd'] = client_df['base_wpb_price_usd'] * (1-client_df['discount']) + client_df['logistics']
    
    client_df['WBP_price_usd'].plot(color='red', linestyle="dashed")
    plt.title(f"Client {client_name} WBP Price")
    plt.ylabel("USD")
    plt.tight_layout()
    plt.savefig(f'{client_name}_wbp_price.png')
    plt.close()

    client_df = client_df.round(2)
    client_df = client_df.reset_index()
    client_df.Date = client_df.Date.dt.strftime('%B %Y')
    
    max_row, max_col = client_df.shape
    client_df.to_excel(writer, sheet_name=client_name, startrow=1, header=False, index=False)
    workbook = writer.book
    worksheet = writer.sheets[client_name]
    column_settings = [{'header': column} for column in client_df.columns]
    worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})
    worksheet.insert_image(max_row + 3, 1, f'{client_name}_wbp_price.png')
    



# Задание 2

Клиенты согласились на ваше предложение. 
Вам нужно сделать написать скрипт, с помощью которого можно автоматом рассчитывать цены для всех клиентов одновременно. 
Важный момент: данные для расчета скрипт должен подтягивать сам.

In [ ]:
clients_path = 'for_clients'
os.makedirs(clients_path, exist_ok=True)

# Задание 3

К вам пришел новый клиент из России. Который использует продукт А и готов регулярно закупать ВБП для того, чтобы не быть привязанным к одному поставщику. Клиент готов забирать ВБП с завода самостоятельно.
На основе тех данных, которые уже есть предложите цену новому клиенту в рублях. 



In [ ]:
#